# Are-MipiA1A2 : Venin

In [345]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt


#class agent :
#    def __init__(self):
#        self.efficacite= 0.1
#        self.spawn = 0.1
#        self.coagulation = 0.1
#        self.perte = 0.1
#
#globule_rouge = agent()
#globule_blanc = agent()
#plaquette = agent()
#venin = agent()
#coagulation = agent()
#anti_coagulant = agent()



#"""Type : Cell = List[float**6]"""





anti_coagulantefficacite = 0.1
globule_blancefficacite = 0.1
veninefficacite = 0.5
plaquettescoagulation = 0.1
veninechange=0.5
globule_blancechange=0.5

In [346]:
def init_matrice(N):
    """Int -> Mat[Cell]
    crée une matrice de taille N*N de type : Mat[list[Gr,Gb,Pl,Ve,Co,Ac,boost]]"""
    matrix = np.zeros((N,N,7))
    mat = np.random.uniform(low=0.878, high=0.985, size=(N,N))
    for i in range(N):
        for j in range(N):
            x = mat[i,j]
            matrix[i,j]=(x , (1 - x)*0.023 , (1 - x)*0.977,0,(1 - x)*0.977,0,1)
    return matrix
            

In [347]:
def init_matrice_venin(N,x,y,Ve):
    matrix = np.zeros((N,N,7))
    mat = np.random.uniform(low=0.878, high=0.985, size=(N,N))
    for i in range(N):
        for j in range(N):
            a = mat[i,j]
            matrix[i,j]=(a , (1 - a)*0.023 , (1 - a)*0.977,0,(1 - a)*0.977,0,1)
    matrix[x,y,3]=Ve
    return matrix

In [348]:
def randomlist(N):
    """Int => list[int]*
    retourne une liste de nombres dans un ordre aléatoire len(list)=N """
    list0 = [i for i in range(N)]
    list1=[]
    rlist=[]
    for i in range(n):
        e = np.random.choice(len(list0))
        rlist.append(list0[e])
        list1=[j for j in list0 if j!=list0[e]]
        list0=list1
    return rlist

In [349]:
def selfinteract(mat,N):
    matrix = mat.copy()
    a = [0 for i in range(7)]
    for i in range(N):
        for j in range(N):
            
            a = mat[i,j].copy()
            
            #globule_rouge+venin
            if a[3] > 0:
                a[0] = a[0] - a[0]*veninefficacite
                if a[3]<0:
                    a[0] = 0
            #globule_blanc+venin
            if a[3] > 0:
                a[3] = a[3] - a[1]*globule_blancefficacite
                a[1] = 0
                a[6] = 10
            #plaquettes+plaquettes
            a[4] = a[4] + a[2]*plaquettescoagulation
            a[2] = a[2] - a[2]/plaquettescoagulation
            #coagulation+anti_coagulant
            if a[5]>0:
                a[4] = a[4] - a[5]*anti_coagulantefficacite
                a[5] = a[5] - a[5]/anti_coagulantefficacite
            #globule_blanc+coagulation
            a[4] = a[4] - a[1]*globule_blancefficacite
            if a[4] <0:
                a[4] = 0
            #venin+anti_coagulant
            if a[3] > 0 and a[5]:
                a[3] = a[3] - a[5]*anti_coagulantefficacite
                a[5] = a[5] - a[5]/anti_coagulantefficacite
            
            #coagulation
            if a[3]>0:
                a[4]=a[4]+a[3] *veninefficacite
                if a[4]>1:
                    a[4]=1
            
            matrix[i,j] = a
    return matrix
            

In [350]:
def interact(mat,i,j,k,l):
    matr = mat.copy()
    matr[i,j,3] = mat[i,j,3] + mat[k,l,3]*veninechange - mat[i,j,3]*veninechange
    matr[k,l,3] = mat[k,l,3] + mat[j,k,3]*veninechange - mat[k,l,3]*veninechange
    
    matr[i,j,1] = mat[i,j,1] + mat[k,l,1]*globule_blancechange - mat[i,j,1]*globule_blancechange
    matr[k,l,1] = mat[k,l,1] + mat[i,j,1]*globule_blancechange - mat[k,l,1]*globule_blancechange
    return matr

In [351]:
def inter_1(N,mat):
    
    
    for i in range(N):
        for j in range(N):
            a = mat[i,j]
            if a[3] > 0 :
                k = np.random.choice(4)
                if k == 0 and i > 0:
                    return interact(mat,i,j,i-1,j)
                if k == 1 and j < N:
                    return interact(mat,i,j,i,j+1)
                if k == 2 and i < N :
                    return interact(mat,i,j,i+1,j)
                if k == 3 and j > 0:
                    return interact(mat,i,j,i,j-1)
            
            return mat

In [352]:
def inter_2(N,mat):
    for i in range(N):
        for j in range(N):
            a =  mat[i,j].copy()
            if a[3] > 0 :
                if i>0:
                    mat = interact(mat,i,j,i-1,j)
                if j<N-1:
                    mat = interact(mat,i,j,i,j+1)
                if i<N-1:
                    mat = interact(mat,i,j,i+1,j)
                if j>0:
                    mat = interact(mat,i,j,i,j-1)
    return mat
            

In [353]:
def lis_co(lis,N):
    """list[mat(cell)]*int => list(mat[float])
    convertie une liste de matrice de cellule en liste de matrice de float (Coagulation)"""
    long=len(lis)
    Lr = np.zeros((long,N,N))
    mat : mat(cell)
    for n in range(long):
        mat = lis[n]
        for i in range(N):
            for j in range(N):
                Lr[n,i,j] = mat[i,j,4]
                
    return Lr
                

In [354]:
def main(N,x,y,Ve,n):
    mat = init_matrice_venin(N,x,y,Ve)
    rlist=np.zeros((n+1,N,N,7))
    rlist[0,:,:,:] = mat.copy()
    for i in range(n):
        mat = inter_2(N,mat)
        mat = selfinteract(mat,N)
        rlist[i+1,:,:,:] = mat[:,:,:]
    return rlist[:,:,:,4]
    

In [355]:
main(3,1,1,2,10)

array([[[0.09517179, 0.04055197, 0.06345944],
        [0.10378948, 0.0249267 , 0.0241136 ],
        [0.08837191, 0.08289464, 0.11267943]],

       [[0.10446492, 0.04453009, 0.06969723],
        [0.17657458, 0.12108982, 0.07135681],
        [0.12836387, 0.12820085, 0.16096411]],

       [[0.01869829, 0.03139793, 0.02033706],
        [0.11544204, 0.13702034, 0.08307005],
        [0.08185985, 0.08737531, 0.09333227]],

       [[0.80571479, 0.38756555, 0.55768023],
        [0.98603866, 0.37099845, 0.30941565],
        [0.82841317, 0.79022026, 1.        ]],

       [[0.02590634, 0.02927398, 0.02810205],
        [0.0298244 , 0.03062976, 0.03037095],
        [0.03025883, 0.03051622, 0.03051622]],

       [[1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ]],

       [[0.02960917, 0.02997137, 0.02986377],
        [0.03003146, 0.03012369, 0.03009955],
        [0.03008629, 0.03011618, 0.03011618]],

       [[1.        ,